In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

In [2]:
from keras.datasets import mnist

In [3]:
X,y=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
img_rows = 28
img_cols = 28
channels = 1

In [5]:
img_shape = (img_rows, img_cols, channels)

In [6]:
from tensorflow import keras
from tensorflow.keras import Model,Sequential
from tensorflow.keras.layers import Dense,LeakyReLU,BatchNormalization,Reshape,Input,Flatten
from tensorflow.keras.optimizers import Adam

In [7]:
def build_generator():
  noise_shape=(100,)

  model = Sequential([
                      Dense(256,input_shape=noise_shape),
                      LeakyReLU(alpha=0.2),
                      BatchNormalization(momentum=0.9),

                      Dense(1024),
                      LeakyReLU(alpha=0.2),
                      BatchNormalization(momentum=0.9),

                      Dense(img_rows*img_cols,activation='tanh'),
                      Reshape(img_shape)
  ])

  noise = Input(shape=noise_shape)
  img = model(noise) 

  return Model(noise,img)

In [8]:
def build_discriminator():
  model = Sequential([
                    Flatten(input_shape = img_shape),
                    Dense(512),
                    LeakyReLU(alpha=0.2),
                    Dense(256),
                    LeakyReLU(alpha=0.2),
                    Dense(1,activation = 'sigmoid')
  ])

  img = Input(shape=img_shape)
  validity = model(img)

  return Model(img,validity)

In [9]:
def train(epochs,batch_size=128,save_interval = 500):

  (X_train, _), (_, _) = mnist.load_data()

  X_train = (X_train.astype(np.float32) - 127.5) / 127.5

  X_train = np.expand_dims(X_train, axis=3)

  half_batch = int(batch_size / 2)

  for epoch in range(epochs):

    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_imgs = X_train[idx]

 
    noise = np.random.normal(0, 1, (half_batch, 100))

    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))

    d_loss = (np.add(d_loss_fake , d_loss_real))/2

    noise = np.random.normal(0, 1, (batch_size, 100))

    valid_y = np.array([1] * batch_size)

    g_loss = combined.train_on_batch(noise, valid_y)

    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if epoch % save_interval == 0:
            save_imgs(epoch)

In [10]:
np.ones((2,1))

array([[1.],
       [1.]])

In [11]:
np.array([1] * 4)

array([1, 1, 1, 1])

In [12]:
np.random.normal(0,1,(2,1))

array([[0.39930381],
       [0.21345038]])

In [13]:
np.random.randint(0,60000,5)

array([34270, 42362, 10841, 35495, 10897])

In [14]:
def save_imgs(epoch):
  r,c = 5,5
  noise = np.random.normal(0, 1, (r * c, 100))
  gen_imgs = generator.predict(noise)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig("mnist_%d.png" % epoch)
  plt.close()

In [15]:
optimizer = Adam(0.0002,0.5)

In [16]:
discriminator = build_discriminator()

In [17]:
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

In [18]:
generator=build_generator()

In [19]:
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

In [20]:
z = Input(shape=(100,))

In [21]:
img = generator(z)

In [22]:
discriminator.trainable=False

In [23]:
valid = discriminator(img)

In [24]:
combined = Model(z, valid)

In [25]:
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [26]:
train(epochs=10000, batch_size=32, save_interval=1000)

Streaming output truncated to the last 5000 lines.
5000 [D loss: 0.502185, acc.: 81.25%] [G loss: 1.036986]
5001 [D loss: 0.665514, acc.: 65.62%] [G loss: 1.036919]
5002 [D loss: 0.592104, acc.: 71.88%] [G loss: 1.054964]
5003 [D loss: 0.570757, acc.: 78.12%] [G loss: 1.055759]
5004 [D loss: 0.619057, acc.: 78.12%] [G loss: 1.017109]
5005 [D loss: 0.729770, acc.: 56.25%] [G loss: 0.940910]
5006 [D loss: 0.616185, acc.: 59.38%] [G loss: 0.956699]
5007 [D loss: 0.603502, acc.: 62.50%] [G loss: 0.969032]
5008 [D loss: 0.602024, acc.: 62.50%] [G loss: 0.954554]
5009 [D loss: 0.534432, acc.: 71.88%] [G loss: 1.012564]
5010 [D loss: 0.613439, acc.: 62.50%] [G loss: 0.951145]
5011 [D loss: 0.522834, acc.: 75.00%] [G loss: 0.992834]
5012 [D loss: 0.678385, acc.: 68.75%] [G loss: 1.066999]
5013 [D loss: 0.606879, acc.: 59.38%] [G loss: 0.959819]
5014 [D loss: 0.690388, acc.: 56.25%] [G loss: 0.934061]
5015 [D loss: 0.590510, acc.: 65.62%] [G loss: 0.916594]
5016 [D loss: 0.515581, acc.: 84.38%]